In [ ]:
import Pkg
Pkg.activate(".")

In [ ]:
Pkg.add(["CairoMakie","LinearAlgebra","Distributions","Convex","SCS","GLMNet"])

In [ ]:
using CairoMakie, LinearAlgebra, Statistics, Distributions, Convex, SCS, GLMNet

In [ ]:
x = LinRange(0,4,100)

In [ ]:
col_list = Vector{Float64}[]
for p in 0:19
    push!(col_list, x .^ p)
end
A = reduce(hcat,col_list)

In [ ]:
# alternative way of constructing the matrix
A2 = zeros(100,20)
for i in 1:100
    for j in 1:20
        A2[i,j] = x[i]^(j-1)
    end
end

In [ ]:
y = x .^2

In [ ]:
Ainv = pinv(A)

In [ ]:
y1 = y .+ rand(Normal(0,0.1),length(y))
b1 = Ainv * y1
barplot(b1)  

In [ ]:
fig = Figure()
ax = Axis(fig[1,1])
lines!(ax,x,y)
plot!(ax,x,y1)
fig

In [ ]:
# Function to fit Least Squares using the pseudo-inverse
function fitlq(A,y)
    Ainv = pinv(A)
    return Ainv * y
end

# Function to fit Lasso regression 
function lasso(A,y,λ1,λ2)
    x = Variable(size(A)[2])
    problem = minimize(norm(A*x-y,2) + λ1*norm(x,1) + λ2*norm(x,2))
    solve!(problem, SCS.Optimizer; silent=true)
    return vec(evaluate(x))
end

In [ ]:
b10ls = lasso(A[:,1:10],y1,0.1,0)  # LASSO fit 
b10lq = fitlq(A[:,1:10],y1)  # Least-Squares fit 

In [ ]:
barplot(b10ls)  # LASSO coefficients
barplot!(b10lq)  # Least-Squares coefficients
current_figure()

In [ ]:
# Error calculation function
function compute_error(y_true, y_pred)
    return mean(abs2, y_true - y_pred)
end

In [ ]:
function run_trials(degree_range, N_trials, N_data)
    errors_ls = []
    errors_lasso = []
    coefficients_ls = []
    coefficients_lasso = []

    # Loop through polynomial degrees
    for degree in degree_range
        err_ls_deg = []
        err_lasso_deg = []
        coeff_ls_deg = []
        coeff_lasso_deg = []
        
        # Multiple realizations of noisy data
        for _ in 1:N_trials
            x, y = generate_data(N_data)
            
            # Least-Squares fitting
            β_ls = fitlq(A[:,1:degree], y1)
            y_pred_ls = A[:,1:degree] * β_ls
            push!(err_ls_deg, compute_error(y1, y_pred_ls))
            push!(coeff_ls_deg, β_ls)

            # LASSO fitting
            β_lasso = lasso(A[:,1:degree], y1, 0.1, 0)
            y_pred_lasso = A[:,1:degree] * β_lasso
            push!(err_lasso_deg, compute_error(y1, y_pred_lasso))
            push!(coeff_lasso_deg, β_lasso)
        end
        
        push!(errors_ls, err_ls_deg)
        push!(errors_lasso, err_lasso_deg)
        push!(coefficients_ls, coeff_ls_deg)
        push!(coefficients_lasso, coeff_lasso_deg)
    end
    
    return errors_ls, errors_lasso, coefficients_ls, coefficients_lasso
end

# Run the trials
degree_range = 1:10
N_trials = 100
N_data = 50
errors_ls, errors_lasso, coefficients_ls, coefficients_lasso = run_trials(degree_range, N_trials, N_data)

In [ ]:
function plot_figure_14a(coefficients_ls, degree_range)
    fig = Figure(resolution=(600, 400))
    ax = Axis(fig[1, 1], title="Least Squares Coefficients", xlabel="Polynomial Degree", ylabel="Coefficient")
    boxplot!(ax, degree_range, coefficients_ls, markersize=8, color=:blue)
    display(fig)
end

# Call the plotting function
plot_figure_14a(coefficients_ls, degree_range)

In [ ]:
function plot_figure_14c(coefficients_lasso, degree_range)
    fig = Figure(resolution=(600, 400))
    ax = Axis(fig[1, 1], title="LASSO Coefficients", xlabel="Polynomial Degree", ylabel="Coefficient")
    boxplot!(ax, degree_range, coefficients_lasso, markersize=8, color=:red)
    display(fig)
end

# Call the plotting function
plot_figure_14c(coefficients_lasso, degree_range)

In [ ]:
function plot_figure_15(errors_ls, errors_lasso, degree_range)
    fig = Figure(resolution=(1200, 400))

    # Least-Squares Error - Figure 15(a)
    ax1 = Axis(fig[1, 1], title="Error (Least Squares)", xlabel="Polynomial Degree", ylabel="Error")
    boxplot!(ax1, degree_range, errors_ls, markersize=8, color=:green)

    # LASSO Error - Figure 15(b)
    ax2 = Axis(fig[1, 2], title="Error (LASSO)", xlabel="Polynomial Degree", ylabel="Error")
    boxplot!(ax2, degree_range, errors_lasso, markersize=8, color=:purple)
    
    display(fig)
end

# Call the plotting function
plot_figure_15(errors_ls, errors_lasso, degree_range)